In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0416 17:03:03.913419 140220760463104 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [3]:
#!pip install bert-tensorflow

In [4]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
import pandas

In [5]:
OUTPUT_DIR = 'tmp'

In [6]:
data1=pd.read_csv("train.csv") #currently in use
data2=pd.read_csv("test_with_solutions.csv")

In [7]:
data1.columns = ['Insult','Date','Comment']
df1=pd.DataFrame([data1['Comment'],data1['Insult']]).T
data2.columns = ['Insult','Date','Comment', 'Usage']
df2=pd.DataFrame([data2['Comment'],data2['Insult']]).T
train=df1
test=df2

In [8]:
DATA_COLUMN = 'Comment'
LABEL_COLUMN = 'Insult'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

In [9]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [10]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0416 17:03:32.746533 140220760463104 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0416 17:03:34.971580 140220760463104 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [11]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [12]:
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 3947


I0416 17:03:39.356843 140220760463104 run_classifier.py:774] Writing example 0 of 3947


INFO:tensorflow:*** Example ***


I0416 17:03:39.359067 140220760463104 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0416 17:03:42.163540 140220760463104 run_classifier.py:466] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0416 17:03:42.164396 140220760463104 run_classifier.py:467] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


I0416 17:03:42.165251 140220760463104 run_classifier.py:468] label: 0 (id = 0)


In [13]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [14]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [15]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [16]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [17]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [18]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_num_worker_replicas': 1, '_task_type': 'worker', '_experimental_distribute': None, '_model_dir': 'tmp', '_save_summary_steps': 100, '_is_chief': True, '_num_ps_replicas': 0, '_train_distribute': None, '_tf_random_seed': None, '_evaluation_master': '', '_save_checkpoints_secs': None, '_device_fn': None, '_keep_checkpoint_max': 5, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_protocol': None, '_global_id_in_cluster': 0, '_master': '', '_save_checkpoints_steps': 500, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f86f45ad278>, '_task_id': 0, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_eval_distribute': None, '_log_step_count_steps': 100}


I0416 17:03:44.089135 140220760463104 estimator.py:201] Using config: {'_num_worker_replicas': 1, '_task_type': 'worker', '_experimental_distribute': None, '_model_dir': 'tmp', '_save_summary_steps': 100, '_is_chief': True, '_num_ps_replicas': 0, '_train_distribute': None, '_tf_random_seed': None, '_evaluation_master': '', '_save_checkpoints_secs': None, '_device_fn': None, '_keep_checkpoint_max': 5, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_protocol': None, '_global_id_in_cluster': 0, '_master': '', '_save_checkpoints_steps': 500, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f86f45ad278>, '_task_id': 0, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_eval_distribute': None, '_log_step_count_steps': 100}


In [19]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [20]:
print('Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Skipping training since max_steps has already saved.


I0416 17:03:44.222442 140220760463104 estimator.py:351] Skipping training since max_steps has already saved.


Training took time  0:00:00.121176


In [21]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [24]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0416 15:42:15.407334 140235964307200 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0416 15:42:18.829857 140235964307200 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving dict for global step 370: auc = 0.85795593, eval_accuracy = 0.885153, f1_score = 0.78500706, false_negatives = 138.0, false_positives = 166.0, global_step = 370, loss = 0.45948133, precision = 0.76976424, recall = 0.8008658, true_negatives = 1788.0, true_positives = 555.0


I0416 15:43:01.117505 140235964307200 estimator.py:1979] Saving dict for global step 370: auc = 0.85795593, eval_accuracy = 0.885153, f1_score = 0.78500706, false_negatives = 138.0, false_positives = 166.0, global_step = 370, loss = 0.45948133, precision = 0.76976424, recall = 0.8008658, true_negatives = 1788.0, true_positives = 555.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 370: tmp/model.ckpt-370


I0416 15:43:03.851156 140235964307200 estimator.py:2039] Saving 'checkpoint_path' summary for global step 370: tmp/model.ckpt-370


{'auc': 0.85795593,
 'eval_accuracy': 0.885153,
 'f1_score': 0.78500706,
 'false_negatives': 138.0,
 'false_positives': 166.0,
 'global_step': 370,
 'loss': 0.45948133,
 'precision': 0.76976424,
 'recall': 0.8008658,
 'true_negatives': 1788.0,
 'true_positives': 555.0}

In [22]:
from sklearn.metrics import roc_auc_score

In [23]:
labels = [0, 1]
predictions = estimator.predict(test_input_fn)
abb=[(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(test['Comment'], predictions)]

INFO:tensorflow:Calling model_fn.


I0416 17:03:59.474957 140220760463104 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0416 17:04:02.560227 140220760463104 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

I0416 17:04:03.232771 140220760463104 saver.py:1270] Restoring parameters from tmp/model.ckpt-370


INFO:tensorflow:Running local_init_op.


I0416 17:04:10.450384 140220760463104 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0416 17:04:10.548501 140220760463104 session_manager.py:493] Done running local_init_op.


In [56]:
pred_y=[]
for i in range(test.shape[0]):
    pred_y.append(abb[i][2])

In [26]:
import numpy as np
y_test=np.array(df2['Insult'])

In [68]:
auc = roc_auc_score(y_test, pred_y)
print('Auc of BERT '+'is '+str(auc))

Auc of BERT is 0.8588871608318897


In [59]:
#pred_y=pd.DataFrame(pred_y)
y_test=pd.DataFrame(df2['Insult'])

In [29]:
y_test

array([0, 0, 1, ..., 0, 1, 0], dtype=object)

In [42]:
pred_y=np.array(pred_y)

In [35]:
pred_y
for a in pred_y:
    if a==1:
        

array([0, 0, 1, ..., 0, 1, 0])

In [49]:
type(y_test)

pandas.core.frame.DataFrame

In [58]:
np.where(y_test > 0.5, 1, 0)

array([[0],
       [0],
       [1],
       ...,
       [0],
       [1],
       [0]])

In [66]:
y_test=y_test.astype(int)